# Polytrope #
Solve the stellar structur equations,
$$\frac{dP(r)}{dr} = -\frac{\rho(r)GM(r)}{r^2}$$
$$\frac{dM(r)}{dr} = 4\pi r^2 \rho(r)$$
with the equation of state $P = K\rho^{\frac{n + 1}{n}}$ where $K$ is a constant and $n$ is the polytropic index. In this case the equations do not have an explicit tempurature dependence and can be conbined to the form,

$$\frac{1}{r^2}\frac{d}{dr}\left( \frac{r^2}{\rho(r)}\frac{dP(r)}{dr} \right) = -4\pi G\rho(r)$$

Written in dimensionless form with $\rho(r) = \rho_c \theta^n(r)$, $P(r) = P_c \theta^{n + 1}(r)$, $\alpha^2 = \frac{(n + 1)P_c}{4\pi G\rho_c^2}$, $\xi = \frac{r}{\alpha}$, where $P_c = K\rho_c^{\frac{n+1}{n}}$, then the result is the dimensionless Lane-Emden equation

$$\frac{1}{\xi^2}\frac{d}{d\xi}\left( \xi^2\frac{d\theta(\xi)}{d\xi} \right) = -\theta^n(\xi)$$

Solutions should satisfy the boundary conditions $\left. \frac{d\theta(\xi)}{\xi} \right|_{\xi = 0} = 0$ and $\left. \theta(\xi) \right|_{\xi = 0} = 1$

In this model a star has radius $R_* = \alpha \xi_0$ and mass $M_* = 4\pi \alpha^3 \rho_c \int_0^{\xi_0} \xi^2 \theta^n(\xi)d\xi$, where $\xi_0$ is the first zero of the function $\theta(\xi)$. Notes from LeBlanc 5.4

## Numerical solution ##
Expanding the Lane-Emden equation,

$$\frac{d^2\theta(\xi)}{d\xi^2} + \frac{2}{\xi}\frac{d\theta(\xi)}{d\xi} = -\theta^n(\xi)$$

and rearanging as a system of first order equaations for use with scipy.integrate.solve_ivp
$$\begin{array}{ll}
\frac{d\theta(\xi)}{d\xi} &= \theta'(\xi) \\
\frac{d\theta'(\xi)}{d\xi} &= - \frac{2}{\xi}\theta'(\xi) - \theta^n(\xi)
\end{array}$$

In [ ]:
%pylab ipympl
import scipy.integrate

# define y = <theta, theta'>, t = xi
def get_dydt(n):
    """Get dydt(t, y)
    
    Parameter
    ---------
    n : float
        Polytropic index
    
    Returns
    -------
    f(t, y)
        The function dydt defining the Lane-Emden equation
    """
    def dydt(t, y):
        """The Lane-Emden equation as a system of first order equations
        
        Parameters
        ----------
        t : float
            Equivelent to the variable xi in the Lane-Emden equation
        y : array
            The vector <theta(xi), theta'(xi)>
        """
        theta, theta_prime = y
        dtheta_dt = theta_prime
        dtheta_prime_dt = -2*theta_prime/t - theta**n
        return [dtheta_dt, dtheta_prime_dt]
    return dydt

t_range = (1e-3, 100)
resolution = 100
n = 5

t = numpy.linspace(*t_range, resolution)

surface_event = lambda t, y:y[0]
surface_event.terminal = True

solution = scipy.integrate.solve_ivp(get_dydt(n), t_range, y0=[1.0, 0.0], events=surface_event, rtol=1e-9, atol=1e-9)

In [ ]:
t = solution["t"]
theta, theta_prime = solution["y"]
r = t/t.max()

ifig = 0
ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, theta**n, label="dencity $y = \\frac{\\rho}{\\rho_c}$")
pyplot.plot(r, theta**(n + 1), label="pressure $y = \\frac{P}{P_c}$")
pyplot.plot(r, theta, label="tempurature $y = \\frac{T}{T_c}$")
pyplot.xlabel("radius $\\frac{r}{R_\\odot}$")
pyplot.ylabel("parameter $y$")
pyplot.ylim(-0.1, 10)
pyplot.legend()
pyplot.show(ifig)